# **_Chess Piece Detection_**

### __*This CNN is supposed to determine what kind of Chess Piece is in the Image*__

<br>

### __*Import Libraries and choose the device*__

In [1]:
# Import of the Libraries
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Torch libraries
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Device config (Pick your set-up)
GPU = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # NVIDIA GPUs
# GPU = torch.device('mps' if torch.has_mps else 'cpu')  # ARM GPUs (M1, M2, ...)
print('Using the Processor') if GPU == torch.device('cpu') else print('Using the Graphics Card')

Using the Graphics Card


<br>

### __*Convert the Images into a Dataset format for PyTorch*__

In [5]:
# Define the directory
FILE_PATH = './data/chess'          # Sub-Folders Location
CATEGORIES = os.listdir(FILE_PATH)  # Turn Sub-Folder names into a list

# Image Size
IMG_SIZE = 85  # All images should be 85*85 anyway


# Convert the Images into a Training & Test Dataset
def create_dataset(file_path, categories, test_ratio=0.2, dtype=np.float32):
    dataset = []
    # Fill the dataset with arrays of img info + label
    for category in categories:
        path = os.path.join(file_path, category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_array_resized = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                img_tensor = torch.from_numpy(np.array(img_array_resized).astype(dtype))
                dataset.append([img_tensor, category])
            except Exception:
                pass

    # Split the dataset into a train & test datasets
    train_data, test_data = train_test_split(dataset, test_size=test_ratio)

    # Turn the datasets into Tensors
    # train_data = torch.from_numpy(np.array(train_data).astype(dtype))
    # test_data = torch.from_numpy(np.array(test_data).astype(dtype))

    return train_data, test_data

#### __*Visual representation of the dataset*__

In [6]:
print_train, print_test = create_dataset(FILE_PATH, CATEGORIES)


#print_test = torch.from_numpy(print_test[0][0])
print(print_test[0])
print(type(print_test[0]))
print(type(print_test[0][0]))
print(type(print_test[0][1]))



print(len(print_train))
print(len(print_test))

train_dct = dict()
for item in print_train:
    try:
        train_dct[item[1]] += 1
    except KeyError:
        train_dct[item[1]] = 1
print(sorted(train_dct.items()))

test_dct = dict()
for item in print_test:
    try:
        test_dct[item[1]] += 1
    except KeyError:
        test_dct[item[1]] = 1
print(sorted(test_dct.items()))

[tensor([[ 93., 167., 160.,  ..., 165., 159., 155.],
        [ 93., 167., 162.,  ..., 172., 172., 163.],
        [ 93., 167., 161.,  ..., 163., 171., 169.],
        ...,
        [111., 169., 188.,  ..., 200., 201., 198.],
        [144., 163., 172.,  ..., 181., 182., 182.],
        [170., 171., 122.,  ..., 115., 118., 118.]]), 'pawn']
<class 'list'>
<class 'torch.Tensor'>
<class 'str'>
413
104
[('bishop', 54), ('king', 22), ('knight', 55), ('pawn', 200), ('queen', 24), ('rook', 58)]
[('bishop', 13), ('king', 12), ('knight', 16), ('pawn', 39), ('queen', 10), ('rook', 14)]


### __*Training Loop*__

In [4]:
# Hyper-parameters
num_epochs = 15
batch_size = 8
learning_rate = 0.001